# RQ4
## Impact of M and w/ (w/o) selection

In [1]:
import pandas as pd

In [10]:
def sample_maxGen(df, maxGen=20):
    df_all = None
    for oid in range(0, max(df['OID'])+1):
        if oid not in df['OID']:
            print('skip: oid = ', oid)
            continue
        df1 = df[df.OID == oid]
        if len(set(df1.pairConsistent.values)) == 2:
            dff = df1[df1.pairConsistent == False]
            if len(dff) >= maxGen:
                df1 = dff.iloc[:maxGen, :]
            else:
                dft = df1[df1.pairConsistent == True].iloc[:maxGen - len(dff), :]
                df1 = pd.concat([dff, dft], axis=0)
        else:
            df1 = df[df.OID == oid].iloc[:min(len(df1), maxGen), :]

        if df_all is None:
            df_all = df1
        else:
            df_all = pd.concat([df_all, df1], axis=0)
            # print(len(df_all))
        # print()
    return df_all

def setup_groundtruth(cr='NeuralCoref', gt_method='Textattack'):  # Stats
    # Because Checklist has all 200 sentences recorded, so we use it as ground truth.
    df = pd.read_csv('../Output/RQ4-Impact/{}/{}.tsv'.format(cr, gt_method), delimiter='\t')
    print(len(set(df['OID'])))
    
    for i in range(0, 1001):
        if i not in set(df['OID']):
            print(i)
    assert len(set(df['OID'])) == 1000

    Groundtruth_N = set(df[df['oriConsistent'] == False]['OID'])
    # print(len(Groundtruth_N))
    return Groundtruth_N

def iterate(folder, cr, select):
    Num_Src = 1000
    Groundtruth_N = setup_groundtruth(cr=cr)
    N = len(Groundtruth_N)
    P = Num_Src - N
    
    
    df = pd.read_csv('{}/{}/Crest.tsv'.format(folder,cr), delimiter='\t')
    

### NeuralCoref | M = 10, 20, 30, 50 | Select = True 

In [12]:
# rq4

Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='NeuralCoref')
N = len(Groundtruth_N)
P = Num_Src - N

select = True

df = pd.read_csv('../Output/RQ4-Impact/NeuralCoref/{}.tsv'.format(method), delimiter='\t')

for m in [10, 20, 30, 50]:
    df_sampled = sample_maxGen(df, maxGen=m)
        
    if select:
        df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
        
    Gen = len(df_sampled)
    Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
        
    hit = 0
    for nid in Groundtruth_N:
        if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
            hit += 1
    
    TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
    FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
    Prec = round(TP / (TP + FP), 2)
    
    if select:
        corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
        corefKeep = Gen - corefChanged
    else:
        corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
        
    corefKeepRatio = round(corefKeep / Gen, 4)

    print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')

1000
1000

Method = Crest
M: 10 | Select: True | Gen: 7947 | Issues: 1314 | Hit: 324 / 465 (69.68) | TP: 1314 | FP: 0 | Prec: 1.0 | CorefKeep: 7947 / 7947 =  1.0
M: 20 | Select: True | Gen: 13635 | Issues: 1457 | Hit: 324 / 465 (69.68) | TP: 1457 | FP: 0 | Prec: 1.0 | CorefKeep: 13635 / 13635 =  1.0
M: 30 | Select: True | Gen: 16611 | Issues: 1488 | Hit: 324 / 465 (69.68) | TP: 1488 | FP: 0 | Prec: 1.0 | CorefKeep: 16611 / 16611 =  1.0
M: 50 | Select: True | Gen: 18698 | Issues: 1519 | Hit: 324 / 465 (69.68) | TP: 1519 | FP: 0 | Prec: 1.0 | CorefKeep: 18698 / 18698 =  1.0


### NeuralCoref | M = 10, 20, 30, 50 | Select = False 

In [16]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='NeuralCoref')
N = len(Groundtruth_N)
P = Num_Src - N

# set coref filter = False
select = False

# read data
df = pd.read_csv('../Output/RQ1-IssueRevealing/generate/NeuralCoref/{}.tsv'.format(method), delimiter='\t')

for m in [10, 20, 30, 50]:
    df_sampled = sample_maxGen(df, maxGen=m)
        
    if select:
        df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
        
    Gen = len(df_sampled)
    Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
        
    hit = 0
    for nid in Groundtruth_N:
        if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
            hit += 1
        
    TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
    FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
    Prec = round(TP / (TP + FP), 2)
    
    if method == 'Crest' and select:
        corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
        corefKeep = Gen - corefChanged
    else:
        corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
        
    corefKeepRatio = round(corefKeep / Gen, 4)

    print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')

1000
1000
M: 10 | Select: False | Gen: 8138 | Issues: 1325 | Hit: 324 / 465 (69.68) | TP: 1325 | FP: 0 | Prec: 1.0 | CorefKeep: 8138 / 8138 =  1.0
M: 20 | Select: False | Gen: 13635 | Issues: 1457 | Hit: 324 / 465 (69.68) | TP: 1457 | FP: 0 | Prec: 1.0 | CorefKeep: 13635 / 13635 =  1.0
M: 30 | Select: False | Gen: 13635 | Issues: 1457 | Hit: 324 / 465 (69.68) | TP: 1457 | FP: 0 | Prec: 1.0 | CorefKeep: 13635 / 13635 =  1.0
M: 50 | Select: False | Gen: 13635 | Issues: 1457 | Hit: 324 / 465 (69.68) | TP: 1457 | FP: 0 | Prec: 1.0 | CorefKeep: 13635 / 13635 =  1.0


### StatsCR | M = 10, 20, 30, 50 | Select = True 

In [ ]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='Stats')
N = len(Groundtruth_N)
P = Num_Src - N

select = True

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'CrestNew',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    # df = pd.read_csv('../NewOutput_Stats/update_{}.tsv'.format(method), delimiter='\t')
    df = pd.read_csv('../NewOutput_NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [1, 10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'CrestNew' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')

### StatsCR | M = 10, 20, 30, 50 | Select = False 

Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='Stats')
N = len(Groundtruth_N)
P = Num_Src - N

select = False

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'CrestNew',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    # df = pd.read_csv('../NewOutput_Stats/update_{}.tsv'.format(method), delimiter='\t')
    df = pd.read_csv('../NewOutput_NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [1, 10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'CrestNew' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')